# Homework 1: Preprocessing and Information Retrieval

Student Name: Saransh Srivastava

Student ID: 1031073

## General info

<b>Due date</b>: Friday, 29 Mar 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 6% of mark for class (with 5% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be using documents from a Wall Street Journal text corpus to create a space efficient inverted index capable of fast TF-IDF query processing.


# 1. Preprocessing (2 marks)

For this homework we will be using documents from a Wall Street Journal text corpus. The corpus can be downloaded with the commands below. Each line contains <i>one</i> document which you should tokenize and stem using tools provided by NLTK. Some of the steps below are already provided whereas others have to be implemented.

<b>Instructions</b>: Run the code below to download documents from a Wall Street Journal text corpus. <b><i>No implementation is needed.</i></b>

In [1]:
import requests
from pathlib import Path

fname = 'wsta_col_20k.gz'
my_file = Path(fname)
if not my_file.is_file():
    url = 'https://trevorcohn.github.io/comp90042/resources/' + fname
    r = requests.get(url)

    # Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)


<b>Instructions</b>: Run the code below to read raw documents, one document per line. <b><i>No implementation is needed.</i></b>

In [2]:
import gzip

raw_docs = []
with gzip.open(fname, 'rt') as f:
    for raw_doc in f:
        raw_docs.append(raw_doc)

print(len(raw_docs))
print(raw_docs[0])

20000
John Blair & Co. is close to an agreement to sell its TV station advertising representation operation and program production unit to an investor group led by James H. Rosenfield, a former CBS Inc. executive, industry sources said. Industry sources put the value of the proposed acquisition at more than $100 million. John Blair was acquired last year by Reliance Capital Group Inc., which has been divesting itself of John Blair's major assets. John Blair represents about 130 local television stations in the placement of national and other advertising. Mr. Rosenfield stepped down as a senior executive vice president of CBS Broadcasting in December 1985 under a CBS early retirement program. Neither Mr. Rosenfield nor officials of John Blair could be reached for comment. 



<b>Instructions</b>: Here, you will do preprocessing. You should *tokenize* each document, *stem* and *lowercase* each token using NLTK `word_tokenize` and `PorterStemmer`, and create a *vocabulary* for all the terms (normalized types). Each term should be assigned a unique ID. Note that we are not doing any stop word removal. The vocabulary should be built as a Python *map*, mapping from all the terms $M$ to their term IDs (integers of $[0..M-1]$). The processing may take a few minutes. 

You may check the section, <i>"For your testing"</i>, below for the expected output.
    
(1 mark)

In [3]:
import nltk
from nltk.tokenize import word_tokenize

###
# Please include the following import lines if word_tokenize is not downloaded due to 'punkt' not installede
#nltk.download('punkt')
###


# processed_docs stores the list of processed docs
processed_docs = []
# vocab contains (term, term id) pairs
vocab = {}
# total_tokens stores the total number of tokens
total_tokens = 0
total_terms = 0

# TODO: iterate over docs, tokenize, stem and add to vocab and assign ID if new token
stemmer = nltk.stem.PorterStemmer()

for raw_doc in raw_docs:
    
    # norm_doc stores the normalized tokens of a doc
    norm_doc = []
    
    ###
    # Your answer BEGINS HERE
    ###
    token_doc = word_tokenize(raw_doc)
    for token_word in token_doc:
        total_tokens += 1
        # total tokens are also representative of IDs of the terms
        term = stemmer.stem(token_word).lower()
        norm_doc.append(term)
        if term not in vocab:
            vocab[term] = total_terms
            total_terms += 1
    ###
    # Your answer ENDS HERE
    ###
    processed_docs.append(norm_doc)

    
print("Number of documents = {}".format(len(processed_docs)))
print("Number of unique terms = {}".format(len(vocab)))
print("Number of tokens = {}".format(total_tokens))

Number of documents = 20000
Number of unique terms = 103193
Number of tokens = 9140697


<b>For your testing:</b>

In [ ]:
assert(len(processed_docs) == 20000)

In [ ]:
assert(len(vocab) > 100000)

<b>Instructions</b>: Now, you should build a Python `Counter` to count the term frequencies of each document. For each document, the counter should map the terms to term frequencies. All the counters (for all documents) should be stored in a list called *doc_term_freqs*.

For example, here is a document:

>the old night keeper keeps the keep in the town. in the big old house in the big old gown. The house in the town had the big old keep where the old night keeper never did sleep. The keeper keeps the keep in the night and keeps in the dark and sleeps in the light.

After the tokenisation and stemming, a counter as below should be built for the document:

`
Counter({'the': 14, 'in': 7, 'keep': 6, 'old': 5, '.': 4, 'night': 3, 'keeper': 3, 'big': 3, 'town': 2, 'hous': 2, 'sleep': 2, 'and': 2, 'gown': 1, 'had': 1, 'where': 1, 'never': 1, 'did': 1, 'dark': 1, 'light': 1})
`

You may check the section, <i>"For your testing"</i>, below for the expected *doc_term_freqs*.

(1 mark)

In [ ]:
from collections import Counter

# doc_term_freqs stores the counters (mapping terms to term frequencies) of all documents
doc_term_freqs = []

# TODO iterate over document and for each document produce the term frequency map and store in the list

###
# Your answer BEGINS HERE
###
for proc_doc in processed_docs:
    doc_term_freqs.append(Counter(proc_doc))
###
# Your answer ENDS HERE
###

print(len(doc_term_freqs))
print(doc_term_freqs[0])
print(doc_term_freqs[100])

<b>For your testing:</b>

In [ ]:
assert(len(doc_term_freqs) == 20000)

In [ ]:
assert(doc_term_freqs[0]["blair"] == 5)

In [ ]:
assert(doc_term_freqs[100]["bank"] == 3)

# 2. Inverted Index (1 mark)

<b>Instructions</b>: Run the code below to create an `InvertedIndex` class using `vocab` and `doc_term_freqs` that you built earlier. <b><i>No implementation is needed.</i></b>

Our `InvertedIndex` class contains <b>six</b> components:

1. The vocabulary `vocab`, which will be used to map query terms to term ids
2. The length of each document,  `doc_len`
3. `doc_ids` is a list indexed by term IDs. For each term ID, it stores a list of document ids of all documents containing that term
4. `doc_term_freqs` is a list indexed by term IDs. For each term ID, it stores a list of document term frequencies $f_{d,t}$ (how often a document $d$ contains the term $t$) of the corresponding documents stored in `doc_ids`
5. `doc_freqs` is a list indexed by term IDs. For each term ID, it stores the document frequency $f_t$ indicating the number of documents containing one or more occurrences of term $t$;
6. Two integers `total_num_docs` and `max_doc_len` store the total number of documents and the maximum document length

These values will be used in the code below. Note that some of these components are for display purposes to verify that your implementation was correctly processing the text collection, but won't be used in TF-IDF scoring.

In [ ]:
class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # this function assumes each integer is stored using 8 bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list) * 8
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list) * 8
        return space_usage
    

invindex = InvertedIndex(vocab, doc_term_freqs)

# print inverted index stats
print("documents = {}".format(invindex.num_docs()))
print("number of terms = {}".format(invindex.num_terms()))
print("longest document length = {}".format(invindex.max_doc_len))
print("uncompressed space usage MiB = {:.3f}".format(invindex.space_in_bytes() / (1024.0 * 1024.0)))

<b>Instructions</b>: Now, you will use the `InvertedIndex` class to compute the TF-IDF similarity scores for the documents given a simple query $Q$.

Here is a simplified formula for computing TF-IDF similarity scores:

\begin{equation*}
Score(Q,d) = \frac{1}{\sqrt{|d|}} \times \sum_{i=1}^q \log(1 + f_{d,t}) * \log( \frac{N}{f_t} ) 
\end{equation*}

where $Q$ corresponds to a query containing $q$ query terms, $\sqrt{|d|}$ corresponds to the length of the document (in words), $f_{d,t}$ corresponds to the frequency of term $t$ in document $d$, $N$ corresponds to the number of documents in the collection, and $f_t$ corresponds to the document frequency of term $t$. All these information are available in the `InvertedIndex` class. Note that the formulation of TF-IDF is a little different to the formula for TF-IDF shown in the lectures. We have adapted the formulation here to allow for a simpler implementation, e.g., avoiding the need for repeated passes over the dataset. (All manner of variants of TF-IDF exist in practise.)

You should implement the `query_tfidf` function. The `query_tfidf` function should take a query and an inverted index and output the top $k$ highest scoring documents. 

For example, here is a query.

> south korea production

Here is a sample result.

> RANK  1  DOCID  176  SCORE  0.426  CONTENT  South Korea rose 1% in February from a year earlier, the
> 
> ...

You may check the section, <i>"For your testing"</i>, below for the expected output.

(1 mark)

In [ ]:
from math import log, sqrt

# given a query and an index returns a list of the k highest scoring documents as tuples containing <docid,score>
def query_tfidf(query, index, k=10):
    
    # scores stores doc ids and their scores
    scores = Counter()
    
    ###
    # Your answer BEGINS HERE
    ###
    # It is assumed that if the query terms are not availab le in the vocabulary
    # then the search engine does not return back any document.
    # Also, if the search query ranks less than k documents in the filtering then only those documents are returned
    
    for q in query:
        if q not in vocab:
            continue
        for doc_id_idx,doc_id in enumerate(index.docids(q)):
            fdt_log = log(1 + index.freqs(q)[doc_id_idx])
            ft_log = log(index.total_num_docs / index.f_t(q))
            scores[doc_id] += (fdt_log * ft_log) / sqrt(index.doc_len[doc_id])
    ###
    # Your answer ENDS HERE
    ###
    
    return scores.most_common(k)


# We output some statistics from our index
query = "south korea productions"
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
results = query_tfidf(stemmed_query, invindex)
for rank, res in enumerate(results):
    # e.g RANK 1 DOCID 176 SCORE 0.426 CONTENT South Korea rose 1% in February from a year earlier, the
    print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

<b>For your testing:</b>

In [ ]:
# Rank 1: DOCID
assert(results[0][0] > 500 and results[0][0] < 3000)

In [ ]:
# Rank 1: SCORE
assert(results[0][1] > 0.5 and results[0][1] < 2)

## 3. Vbyte compression and decompression (2 marks)

Next, we will reduce the space usage of the inverted index by compression. We will <i>compress</i> the `doc_ids` and `doc_term_freqs` lists in the inverted index using <b>vbyte</b> compression.

<b>Instructions</b>: You should implement two methods to perform vbyte compression and decompression as described in the lecture slides. The method signatures are provided below. 

- The first method `vbyte_encode(num)` should receive a number as an integer and produces a list of output bytes encoding the number. 
- The second method `vbyte_decode(input_bytes, idx)` should receive a list of input bytes and an offset into that list where the decompression should start. It returns the decoded number and the number of bytes consumed to decode the number.

(1 mark)

In [ ]:
def vbyte_encode(num):

    # out_bytes stores a list of output bytes encoding the number
    out_bytes = []
    
    ###
    # Your answer BEGINS HERE
    ###
    while num >= 128:
        out_bytes.append(num %128)
        num = num >> 7
    out_bytes.append(num+128)
    ###
    # Your answer ENDS HERE
    ###
    
    return out_bytes


def vbyte_decode(input_bytes, idx):
    
    # x stores the decoded number
    x = 0
    # consumed stores the number of bytes consumed to decode the number
    consumed = 0

    ###
    # Your answer BEGINS HERE
    ###
    shift = 0
    decode_num = input_bytes[idx]
    while decode_num < 128:
        idx += 1
        x = x ^ (decode_num << shift)
        shift += 7
        decode_num = input_bytes[idx]
        consumed += 1
    x = x ^ ((decode_num-128) << shift)
    consumed += 1
    ###
    # Your answer ENDS HERE
    ###
    
    return x, consumed


<b>For your testing:</b>

In [ ]:
# As a sanity check, we ensure that compression and decompression work correctly:
for num in range(0, 123456):
    vb = vbyte_encode(num)
    dec, decoded_bytes = vbyte_decode(vb, 0)
    assert(num == dec)
    assert(decoded_bytes == len(vb))

<b>Instructions</b>: Now, you should modify the `InvertedIndex` class to support compression.

Your task here is to implement the compression of the `doc_ids` and `doc_term_freqs` lists using the `vbyte_encode` function implemented earlier. Note that the `doc_ids` have to be gap encoded as described in the lecture slides. A helper function `decompress_list` is provided to allow easy decompression of the lists. 

(1 mark)

In [ ]:
def decompress_list(input_bytes, gapped_encoded):
    res = []
    prev = 0
    idx = 0
    while idx < len(input_bytes):
        dec_num, consumed_bytes = vbyte_decode(input_bytes, idx)
        idx += consumed_bytes
        num = dec_num + prev
        res.append(num)
        if gapped_encoded:
            prev = num
    return res

class CompressedInvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

        # TODO NOW WE COMPRESS THE LISTS
        
        ###
        # Your answer BEGINS HERE
        ###
        for doc_termId_Idx,doc_termId in enumerate(self.doc_ids):
            prev = 0
            docId_list = []
            for docId in doc_termId:
                for id in vbyte_encode(docId - prev):
                    docId_list.append(id)
                prev = docId
            self.doc_ids[doc_termId_Idx] = docId_list
        
        for doc_termId_idx,doc_termId in enumerate(self.doc_term_freqs):
            term_freq_list = []
            for docTF in doc_termId:
                for freq in vbyte_encode(docTF):
                    term_freq_list.append(freq)
            self.doc_term_freqs[doc_termId_idx] = term_freq_list
        ###
        # Your answer ENDS HERE
        ###
    
    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        # We decompress
        return decompress_list(self.doc_ids[term_id], True)

    def freqs(self, term):
        term_id = self.vocab[term]
        # We decompress
        return decompress_list(self.doc_term_freqs[term_id], False)

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # this function assumes the integers are now bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list)
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list)
        return space_usage


# We output the same statistics as before to ensure we still store the same data but now use much less space
compressed_index = CompressedInvertedIndex(vocab, doc_term_freqs)

print("documents = {}".format(compressed_index.num_docs()))
print("unique terms = {}".format(compressed_index.num_terms()))
print("longest document = {}".format(compressed_index.max_doc_len))
print("compressed space usage MiB = {:.3f}".format(compressed_index.space_in_bytes() / (1024.0 * 1024.0)))

<b>For your testing:</b>

In [ ]:
# Additionally we want to ensure that the index still returns the same results as before
query = "south korea productions"
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
comp_results = query_tfidf(stemmed_query, compressed_index)
for rank, res in enumerate(comp_results):
    print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))